In [1]:
!pip install langchain_openai
!pip install langchain-community
!pip install pypdf
!pip install faiss-cpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.7 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.1.53 which is incompatible.
langchain-openai 0.2.10 requires langchain-core<0.4.0,>=0.3.21, but you have langchain-core 0.1.53 which is incompatible.
langchain-text-splitters 0.3.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.1.53 which is incompatible.



  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21


In [ ]:
#사용 환경 준비
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [12]:
# 데이터 로딩 및 chunking 처리
def load_chunk_data(file_path, chunk_size=100, chunk_overlap=50):
    loader = CSVLoader(file_path=file_path)
    documents = loader.load()
# 텍스트 분할기 설정 (chunk_size와 chunk_overlap 조정 가능)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
# 데이터를 작은 덩어리로 나누기
    split_docs = text_splitter.split_documents(documents)
    return split_docs



In [13]:
# 문화제 데이터와 여행지 데이터 로딩 및 chunk 처리
culture_data = load_chunk_data("final(culture).csv")
place_data = load_chunk_data("final(place).csv")

In [14]:
# OpenAI LLM 설정하기(온도는 어느정도로 하는게 낫나요? 사실 위 청크 사이즈도..)
llm = OpenAI(temperature=0.7)

In [17]:
# 첫 번째 prompt : 문화제에 관련된 답변을 찾기
culture_prompt = """
사용자의 질문에 대한 내용을 기반으로, 문화제와 관련된 정보를 찾아 주세요.
예시 질문: "이 지역의 문화제는 무엇이 있나요?"
"""

culture_chain = LLMChain(llm=llm, prompt=PromptTemplate(input_variables=[], template=culture_prompt))


In [18]:
# 첫 번째 promt : 문화제에 관련된 답변
def get_culture_result(user_question, chunked_data):
    # chunked_data에서 유의미한 문서 찾기 (간단한 예시로 첫 번째 덩어리 사용)
    culture_result = culture_chain.run(input=user_question)
    return culture_result

In [19]:
# 두 번째 prompt : 여행지에 관련된 답변을 찾기
place_prompt = """
사용자가 언급한 지역에 대한 여행지 정보를 제공해 주세요.
예시 질문: "이 지역에서 가볼 만한 여행지는 어디인가요?"
"""

place_chain = LLMChain(llm=llm, prompt=PromptTemplate(input_variables=[], template=place_prompt))


In [20]:
# 두 번째 프롬프트에 대한 답변 (여행지)
def get_place_result(user_question, region_name, chunked_data):
    # chunked_data에서 유의미한 문서 찾기 (간단한 예시로 첫 번째 덩어리 사용)
    place_result = place_chain.run(input=user_question)
    return place_result

In [21]:
# 사용자가 질문한 내용 (이거 동, 구도 ????? )
user_question = "부산에서 어떤 문화제가 유명하나요?"
region_name = "부산"

In [1]:
# 첫 번째 결과: 문화제에 대한 응답
result1 = get_culture_result(user_question, culture_data)

# 두 번째 결과: 여행지에 대한 응답
result2 = get_place_result(user_question, region_name, place_data)

NameError: name 'get_culture_result' is not defined

In [23]:
# 두 결과 합치기
final_result = f"문화제 관련 정보: {result1}\n\n여행지 관련 정보: {result2}"

In [24]:
# 최종 결과 출력
print(final_result)

문화제 관련 정보: 
이 지역의 문화제는 다양한 종류가 있을 수 있습니다. 일반적으로 문화제는 지역의 역사, 전통, 예술, 음악, 음식 등을 기념하거나 선보이는 행사로 분류할 수 있습니다.

예를 들어, 한국의 대표적인 문화제 중 하나인 "전주 한옥마을 축제"는 전라북도 전주시에서 매년 10월에 개최되며, 전통 음식, 한복 체험, 전통 공연 등 다양한 문화 프로그램을 즐길 수 있습니다.

또한, "서울 국제음악제"는 매년 9월에 서울시에서 개최되는 대규모 음악 축제로, 국내외 유명 음악가들의

여행지 관련 정보: 
"이 지역에서 가볼 만한 여행지는 어디인가요?"

저는 해당 지역의 관광지, 문화 시설, 자연 경관 등 다양한 여행지 정보를 제공할 수 있습니다. 아래는 해당 지역에서 추천드리는 몇 가지 여행지 예시입니다.

1. A씨티 랜드마크: 이곳은 지역의 대표적인 랜드마크로, 도시의 역사와 문화를 알 수 있는 곳입니다. 또한 전망대에서는 멋진 전망을 즐길 수 있습니다.
2. B공원: 이곳은 자연 그대로의 아름다움을 감상할 수 있는 공원으로, 산책로, 수목원, 연못 등 다양한 시설을 갖추고 있습니다.
3. C박물관:
